In [2]:
import pandas as pd
import numpy as np
import time
import os
import multiprocessing as mp # for speeding up some process
import sys
sys.path.append('../py_model')
from utils import init_logging
import logging 
from scipy.cluster import hierarchy
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [16, 10] # customizing


In [2]:
# loading data
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
# handling with infinity
df.replace([np.inf, -np.inf], np.nan, inplace = True)
print (df.shape)


(356251, 281)
(356251, 280)


In [3]:
# check transpose
df_transposed = df.T
print (df_transposed.shape)
df_transposed

(280, 356251)


,0,1,2,3,4,5,6,7,8,9,...,356241,356242,356243,356244,356245,356246,356247,356248,356249,356250
AMT_ANNUITY,24700.500000,3.569850e+04,6750.000000,2.968650e+04,21865.500000,27517.500000,4.130100e+04,4.207500e+04,3.382650e+04,20250.000000,...,46660.500000,7.411500e+03,48888.000000,2.857050e+04,23112.000000,17473.500000,31909.500000,33205.500000,25128.000000,2.470950e+04
AMT_CREDIT,406597.500000,1.293502e+06,135000.000000,3.126825e+05,513000.000000,490495.500000,1.560726e+06,1.530000e+06,1.019610e+06,405000.000000,...,835686.000000,1.090080e+05,500490.000000,3.600000e+05,252022.500000,412560.000000,622413.000000,315000.000000,450000.000000,3.127680e+05
AMT_GOODS_PRICE,351000.000000,1.129500e+06,135000.000000,2.970000e+05,513000.000000,454500.000000,1.395000e+06,1.530000e+06,9.135000e+05,405000.000000,...,765000.000000,9.000000e+04,450000.000000,3.600000e+05,217561.500000,270000.000000,495000.000000,315000.000000,450000.000000,2.700000e+05
AMT_REQ_CREDIT_BUREAU_QRT,0.000000,0.000000e+00,0.000000,NaN,0.000000,1.000000,1.000000e+00,0.000000e+00,0.000000e+00,NaN,...,0.000000,NaN,0.000000,0.000000e+00,2.000000,0.000000,NaN,3.000000,0.000000,1.000000e+00
APARTMENTS_MEDI,0.025000,9.680000e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.957000e-01,0.043700,1.228000e-01,0.022900,NaN,NaN,0.112400,0.164500,NaN
CODE_GENDER,0.000000,1.000000e+00,0.000000,1.000000e+00,0.000000,0.000000,1.000000e+00,0.000000e+00,1.000000e+00,0.000000,...,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000e+00
DAYS_BIRTH,-9461.000000,-1.676500e+04,-19046.000000,-1.900500e+04,-19932.000000,-16941.000000,-1.377800e+04,-1.885000e+04,-2.009900e+04,-14469.000000,...,-11387.000000,-2.299500e+04,-21780.000000,-1.939700e+04,-11708.000000,-19970.000000,-11186.000000,-15922.000000,-13968.000000,-1.396200e+04
DAYS_EMPLOYED,-637.000000,-1.188000e+03,-225.000000,-3.039000e+03,-3038.000000,-1588.000000,-3.130000e+03,-4.490000e+02,NaN,-2019.000000,...,-846.000000,NaN,NaN,-1.190000e+02,-369.000000,-5169.000000,-1149.000000,-3037.000000,-2731.000000,-6.330000e+02
DAYS_ID_PUBLISH,-2120.000000,-2.910000e+02,-2531.000000,-2.437000e+03,-3458.000000,-477.000000,-6.190000e+02,-2.379000e+03,-3.514000e+03,-3992.000000,...,-2530.000000,-3.988000e+03,-5249.000000,-2.945000e+03,-4178.000000,-3399.000000,-3003.000000,-1504.000000,-1364.000000,-4.220000e+03
DAYS_LAST_PHONE_CHANGE,-1134.000000,-8.280000e+02,-815.000000,-6.170000e+02,-1106.000000,-2536.000000,-1.562000e+03,-1.070000e+03,0.000000e+00,-1673.000000,...,-655.000000,-1.707000e+03,-611.000000,-1.252000e+03,-987.000000,-684.000000,0.000000,-838.000000,-2308.000000,-3.270000e+02


In [87]:
df_transposed.dropna()

,0,1,2,3,4,5,6,7,8,9,...,356241,356242,356243,356244,356245,356246,356247,356248,356249,356250
AMT_CREDIT,406597.500000,1.293502e+06,135000.000000,312682.500000,513000.000000,490495.500000,1.560726e+06,1.530000e+06,1.019610e+06,405000.000000,...,835686.000000,109008.000000,500490.000000,360000.000000,252022.500000,412560.000000,622413.000000,315000.000000,450000.000000,312768.000000
CODE_GENDER,0.000000,1.000000e+00,0.000000,1.000000,0.000000,0.000000,1.000000e+00,0.000000e+00,1.000000e+00,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
DAYS_BIRTH,-9461.000000,-1.676500e+04,-19046.000000,-19005.000000,-19932.000000,-16941.000000,-1.377800e+04,-1.885000e+04,-2.009900e+04,-14469.000000,...,-11387.000000,-22995.000000,-21780.000000,-19397.000000,-11708.000000,-19970.000000,-11186.000000,-15922.000000,-13968.000000,-13962.000000
DAYS_ID_PUBLISH,-2120.000000,-2.910000e+02,-2531.000000,-2437.000000,-3458.000000,-477.000000,-6.190000e+02,-2.379000e+03,-3.514000e+03,-3992.000000,...,-2530.000000,-3988.000000,-5249.000000,-2945.000000,-4178.000000,-3399.000000,-3003.000000,-1504.000000,-1364.000000,-4220.000000
FLAG_DOCUMENT_11,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FLAG_DOCUMENT_18,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FLAG_DOCUMENT_3,1.000000,1.000000e+00,0.000000,1.000000,0.000000,1.000000,0.000000e+00,1.000000e+00,1.000000e+00,0.000000,...,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
FLAG_DOCUMENT_8,0.000000,0.000000e+00,0.000000,0.000000,1.000000,0.000000,1.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FLAG_OWN_CAR,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,1.000000e+00,1.000000e+00,0.000000e+00,0.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
FLAG_PHONE,1.000000,1.000000e+00,1.000000,0.000000,0.000000,1.000000,1.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000


# The k-prototypes algorithm combines k-modes and k-means and is able to cluster mixed numerical / categorical data.

# reference: https://github.com/nicodv/kmodes/blob/master/kmodes/kprototypes.py

# It is a common practice to apply PCA (principal component analysis) before a clustering algorithm (such as k-means). It is believed that it improves the clustering results in practice (noise reduction). 
### reference: https://stats.stackexchange.com/questions/183236/what-is-the-relation-between-k-means-clustering-and-pca

In [65]:
# pca
from sklearn.decomposition import PCA
pca = PCA(n_components=150, svd_solver='full')
pca.fit(df_transposed)                 



[9.99999132e-01 4.29842661e-07 2.27402633e-07 7.62337026e-08
 4.82728654e-08 3.53602026e-08 1.67597954e-08 9.67252409e-09
 7.10275195e-09 5.68050173e-09 3.23487577e-09 1.97492128e-09
 1.58765136e-09 9.78058557e-10 7.61081928e-10 4.67401358e-10
 4.10743421e-10 3.78166969e-10 3.40837684e-10 3.10050733e-10
 3.05795724e-10 2.65531356e-10 1.70186062e-10 1.49926041e-10
 1.32879934e-10 1.06497464e-10 9.47996568e-11 5.81658367e-11
 4.41065434e-11 3.80147149e-11 1.88966854e-11 1.38444526e-11
 1.17910327e-11 6.56370307e-12 5.47931831e-12 4.03097333e-12
 3.61450789e-12 1.85737192e-12 1.53299662e-12 1.13637715e-12
 1.07586708e-12 1.02477325e-12 9.71034776e-13 6.42482256e-13
 4.97696722e-13 4.81217826e-13 4.29886564e-13 2.81741320e-13
 2.07523694e-13 1.64082694e-13 1.58089742e-13 1.45841618e-13
 1.44954815e-13 1.41341177e-13 1.15928314e-13 4.83959678e-14
 3.65412393e-14 3.38495062e-14 3.24418732e-14 2.92027705e-14
 1.48117451e-14 1.25953749e-14 9.67834791e-15 8.55430283e-15
 7.32475480e-15 5.514036

In [78]:
pca.explained_variance_ratio_[:100].sum()

1.0

In [79]:
df_transposed_reduction = pca.fit_transform(df_transposed)

In [80]:
df_transposed_reduction.shape

(280, 150)

In [81]:
import numpy as np
from sklearn.cluster import KMeans

km = KMeans(n_clusters = 2, n_init=5, verbose=1, n_jobs = 32)

clusters = km.fit(df_transposed_reduction)


Initialization complete
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 2.196625996051963e+20
Initialization complete
start iteration
done sorting
start iteration
start iteration
end inner loop
done sorting
Iteration 1, inertia 2.196625996051963e+20
end inner loop
center shift 0.000000e+00 within tolerance 1.207071e+17
Iteration 0, inertia 2.196625996051963e+20
done sorting
start iteration
end inner loop
done sorting
end inner loop
Iteration 0, inertia 2.196625996051963e+20
Iteration 1, inertia 2.196625996051963e+20
start iteration
done sorting
center shift 0.000000e+00 within tolerance 1.207071e+17
end inner loop
Iteration 1, inertia 2.196625996051963e+20
center shift 0.000000e+00 within tolerance 1.207071e+17
Initialization complete
Initialization complete
start iteration
done sorting
start iteration
end inner loop
done sorting
end inner loop
Iteration 0, inertia 2.196625996051963e+20
start iteration
Iteration 0, inertia 2.196625996051963e+20


In [82]:
# Print Predict the closest cluster
km.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [83]:
feature_clustering = pd.DataFrame({
    'features':df_transposed.index.tolist(),
    'cluster': km.labels_
})
feature_clustering.cluster.value_counts()

0    278
1      2
Name: cluster, dtype: int64

In [56]:
feature_clustering[feature_clustering.cluster == 1]

,features,cluster
267,CC_AMT_RECIVABLE_VAR,1
272,CC_AMT_TOTAL_RECEIVABLE_VAR,1


In [91]:
len(df)

356251

# using k-means algorithm cannot effectively cluster the features.
# They reason probably is that we replace nan w zero(I have to, otherwise I cannot feed the data into k-means+PCA)
# So, we switch to k-mode,  k-modes is used for clustering categorical variables. It defines clusters based on the number of matching categories between data points. (This is in contrast to the more well-known k-means algorithm, which clusters numerical data based on Euclidean distance.) 

In [3]:
import numpy as np
from kmodes.kmodes import KModes
import pandas as pd
# loading data
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
# handling with infinity
df.replace([np.inf, -np.inf], np.nan, inplace = True)
print (df.shape)
# preprocessing for clusting
df.replace(np.nan, 0, inplace = True)

# check transpose
df_transposed = df.T
print (df_transposed.shape)

# random categorical data
data = np.random.choice(20, (100, 10))

km = KModes(n_clusters=4, init='Cao', n_init=5, verbose=1)

clusters = km.fit_predict(df_transposed)


(356251, 281)
(356251, 280)
(280, 356251)
Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 49, cost: 49495810.0
Run 1, iteration: 2/100, moves: 3, cost: 49495810.0


In [10]:
# Print the cluster centroids
feature_clustering = pd.DataFrame({
    'features':df_transposed.index.tolist(),
    'cluster': km.labels_
})
feature_clustering.cluster.value_counts()

0    244
1     24
2     11
3      1
Name: cluster, dtype: int64

In [12]:
#feature_clustering.to_csv('../output/feature_clustering.csv', index = False)
feature_clustering = pd.read_csv('../output/feature_clustering.csv')
feature_clustering.cluster.value_counts()

0    244
1     24
2     11
3      1
Name: cluster, dtype: int64

In [13]:
# reload again for filling
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
# handling with infinity
df.replace([np.inf, -np.inf], np.nan, inplace = True)

(356251, 281)


In [16]:
for f_in_same_cluster in feature_clustering[feature_clustering.cluster == 0].features.tolist():
    print (f_in_same_cluster)
    print (df[f_in_same_cluster].count())

AMT_ANNUITY
356215
AMT_GOODS_PRICE
355973
AMT_REQ_CREDIT_BUREAU_QRT
308683
APARTMENTS_MEDI
176304
DAYS_EMPLOYED
291603
DAYS_ID_PUBLISH
356251
DAYS_LAST_PHONE_CHANGE
356250
DEF_30_CNT_SOCIAL_CIRCLE
355201
DEF_60_CNT_SOCIAL_CIRCLE
355201
ENTRANCES_MEDI
177845
EXT_SOURCE_1
162343
EXT_SOURCE_2
355583
EXT_SOURCE_3
286618
FLAG_DOCUMENT_11
356251
FLAG_DOCUMENT_18
356251
FLAG_DOCUMENT_8
356251
FLAG_OWN_CAR
356251
FLAG_PHONE
356251
FLAG_WORK_PHONE
356251
LIVE_CITY_NOT_WORK_CITY
356251
LIVINGAREA_MEDI
178350
OBS_60_CNT_SOCIAL_CIRCLE
355201
OWN_CAR_AGE
121012
REGION_POPULATION_RELATIVE
356251
REG_CITY_NOT_LIVE_CITY
356251
SK_ID_CURR
356251
TARGET
307507
YEARS_BEGINEXPLUATATION_MEDI
183389
NUM_INSTALMENTS
356215
DIFF_CREDIT_AND_GOODS_RATIO
355973
NEW_DOC_IND_STD
356251
NEW_DOC_IND_KURT
356251
AMT_INCOME_TOTAL_BY_CODE_GENDER_NAME_EDUCATION_TYPE
356251
AMT_REQ_CREDIT_BUREAU_YEAR_BY_NAME_EDUCATION_TYPE_OCCUPATION_TYPE
244256
APARTMENTS_AVG_BY_NAME_EDUCATION_TYPE_OCCUPATION_TYPE
244254
BASEMENTAREA_AV

In [15]:
for f_in_same_cluster in feature_clustering[feature_clustering.cluster == 1].features.tolist():
    print (f_in_same_cluster)
    print (df[f_in_same_cluster].count())

CODE_GENDER
356251
FLAG_DOCUMENT_3
356251
REGION_RATING_CLIENT
356251
REGION_RATING_CLIENT_W_CITY
356251
INCOME_TO_GOODS_PRICE_RATIO
355973
NAME_CONTRACT_TYPE_Cash loans
356251
NAME_EDUCATION_TYPE_Secondary / secondary special
356251
NAME_FAMILY_STATUS_Married
356251
NAME_HOUSING_TYPE_House / apartment
356251
NAME_INCOME_TYPE_Working
356251
PREV_APP_CREDIT_PERC_MAX
338600
PREV_APP_CREDIT_PERC_MEAN
338600
PREV_HUMAN_EVAL_MEAN
338853
PREV_HUMAN_EVAL_SUM
338853
PREV_NAME_CONTRACT_TYPE_Consumer loans_MEAN
338853
PREV_NAME_CONTRACT_STATUS_Approved_MEAN
338853
PREV_CODE_REJECT_REASON_XAP_MEAN
338853
PREV_NAME_PORTFOLIO_POS_MEAN
338853
APPROVED_APP_CREDIT_PERC_MIN
337685
APPROVED_HUMAN_EVAL_SUM
337694
POS_TIME_DECAYED_UNPAYED_RATIO_MAX
337220
POS_NAME_CONTRACT_STATUS_Active_MEAN
337248
INSTAL_PAYMENT_PERC_MEAN_MEAN
339555
INSTAL_PAYMENT_PERC_SUM_SUM
339564


In [20]:
for f_in_same_cluster in feature_clustering[feature_clustering.cluster == 2].features.tolist():
    print (df[f_in_same_cluster].count())

356251
356251
244257
244257
244257
244257
244257
244257
244254
244257
244257


# imputation - by Matrix Completion

In [41]:
from fancyimpute import BiScaler
# loading data
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
# handling with infinity
df.replace([np.inf, -np.inf], np.nan, inplace = True)
print (df.shape)


Using TensorFlow backend.
/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


(356251, 281)
(356251, 280)


In [42]:
X_missing = df[feature_clustering[feature_clustering.cluster == 1].features.tolist()]
X_missing.shape

(356251, 24)

In [44]:
X_filled1 = BiScaler().fit_transform(X_missing.values)

[BiScaler] Initial log residual value = 20.337300
[BiScaler] Iter 1: log residual = 9.581770, log improvement ratio=10.755530
[BiScaler] Iter 2: log residual = 9.021605, log improvement ratio=0.560164
[BiScaler] Iter 3: log residual = 8.049758, log improvement ratio=0.971847
[BiScaler] Iter 4: log residual = 6.492047, log improvement ratio=1.557711
[BiScaler] Iter 5: log residual = 4.785250, log improvement ratio=1.706797
[BiScaler] Iter 6: log residual = 3.030208, log improvement ratio=1.755042
[BiScaler] Iter 7: log residual = 1.260778, log improvement ratio=1.769430
[BiScaler] Iter 8: log residual = -0.510901, log improvement ratio=1.771679
[BiScaler] Iter 9: log residual = -2.279761, log improvement ratio=1.768860
[BiScaler] Iter 10: log residual = -4.042889, log improvement ratio=1.763128
[BiScaler] Iter 11: log residual = -5.797682, log improvement ratio=1.754794
[BiScaler] Iter 12: log residual = -7.541091, log improvement ratio=1.743409
[BiScaler] Iter 13: log residual = -9.269

In [47]:
X_filled1 = pd.DataFrame(X_filled1, columns = X_missing.columns)

In [48]:
X_filled1.shape

(356251, 24)

In [49]:
# reload again for filling
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
# handling with infinity
df.replace([np.inf, -np.inf], np.nan, inplace = True)

(356251, 281)


In [50]:
for f_in_same_cluster in X_filled1.columns.tolist():
    print ('featurs', f_in_same_cluster)
    df[f_in_same_cluster] = X_filled1[f_in_same_cluster].tolist()

featurs CODE_GENDER
featurs FLAG_DOCUMENT_3
featurs REGION_RATING_CLIENT
featurs REGION_RATING_CLIENT_W_CITY
featurs INCOME_TO_GOODS_PRICE_RATIO
featurs NAME_CONTRACT_TYPE_Cash loans
featurs NAME_EDUCATION_TYPE_Secondary / secondary special
featurs NAME_FAMILY_STATUS_Married
featurs NAME_HOUSING_TYPE_House / apartment
featurs NAME_INCOME_TYPE_Working
featurs PREV_APP_CREDIT_PERC_MAX
featurs PREV_APP_CREDIT_PERC_MEAN
featurs PREV_HUMAN_EVAL_MEAN
featurs PREV_HUMAN_EVAL_SUM
featurs PREV_NAME_CONTRACT_TYPE_Consumer loans_MEAN
featurs PREV_NAME_CONTRACT_STATUS_Approved_MEAN
featurs PREV_CODE_REJECT_REASON_XAP_MEAN
featurs PREV_NAME_PORTFOLIO_POS_MEAN
featurs APPROVED_APP_CREDIT_PERC_MIN
featurs APPROVED_HUMAN_EVAL_SUM
featurs POS_TIME_DECAYED_UNPAYED_RATIO_MAX
featurs POS_NAME_CONTRACT_STATUS_Active_MEAN
featurs INSTAL_PAYMENT_PERC_MEAN_MEAN
featurs INSTAL_PAYMENT_PERC_SUM_SUM


In [51]:
df.shape

(356251, 280)

In [52]:
import os
#-------------------
# save
#-------------------
output_path = '../features/filled_by_matrix_completion'
if not os.path.isdir(output_path):
    os.mkdir(output_path)

df.to_hdf(
    os.path.join(output_path, 'base_featurs_filled_by_matrix_cluster_1.h5'), 
    'base_featurs_filled_by_matrix_cluster_1')


In [53]:
from fancyimpute import BiScaler
# loading data
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
# handling with infinity
df.replace([np.inf, -np.inf], np.nan, inplace = True)
print (df.shape)
X_missing = df[feature_clustering[feature_clustering.cluster == 2].features.tolist()]
print (X_missing.shape)
X_filled2 = BiScaler().fit_transform(X_missing.values)

(356251, 281)
(356251, 280)
(356251, 11)
[BiScaler] Initial log residual value = 37.680961
[BiScaler] Iter 1: log residual = 12.191562, log improvement ratio=25.489399
[BiScaler] Iter 2: log residual = 10.427514, log improvement ratio=1.764048
[BiScaler] Iter 3: log residual = 8.983707, log improvement ratio=1.443807
[BiScaler] Iter 4: log residual = 7.851408, log improvement ratio=1.132299
[BiScaler] Iter 5: log residual = 7.446411, log improvement ratio=0.404997
[BiScaler] Iter 6: log residual = 6.889206, log improvement ratio=0.557205
[BiScaler] Iter 7: log residual = 6.410323, log improvement ratio=0.478883
[BiScaler] Iter 8: log residual = 6.006865, log improvement ratio=0.403458
[BiScaler] Iter 9: log residual = 5.727612, log improvement ratio=0.279253
[BiScaler] Iter 10: log residual = 5.868310, log improvement ratio=-0.140698


In [54]:
X_filled2 = pd.DataFrame(X_filled2, columns = X_missing.columns)

In [55]:
# reload again for filling
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
# handling with infinity
df.replace([np.inf, -np.inf], np.nan, inplace = True)
for f_in_same_cluster in X_filled2.columns.tolist():
    print ('featurs', f_in_same_cluster)
    df[f_in_same_cluster] = X_filled2[f_in_same_cluster].tolist()

(356251, 281)
featurs AMT_CREDIT
featurs DAYS_BIRTH
featurs AMT_CREDIT_BY_NAME_EDUCATION_TYPE_OCCUPATION_TYPE
featurs DAYS_BIRTH_BY_OCCUPATION_TYPE
featurs DAYS_BIRTH_BY_CODE_GENDER_OCCUPATION_TYPE
featurs DAYS_BIRTH_BY_REG_CITY_NOT_WORK_CITY_OCCUPATION_TYPE
featurs DAYS_BIRTH_BY_NAME_INCOME_TYPE_OCCUPATION_TYPE
featurs AMT_CREDIT_BY_NAME_INCOME_TYPE_OCCUPATION_TYPE
featurs AMT_GOODS_PRICE_BY_NAME_INCOME_TYPE_OCCUPATION_TYPE
featurs DAYS_BIRTH_BY_ORGANIZATION_TYPE_OCCUPATION_TYPE
featurs DAYS_REGISTRATION_BY_ORGANIZATION_TYPE_OCCUPATION_TYPE


In [56]:
import os
#-------------------
# save
#-------------------
output_path = '../features/filled_by_matrix_completion'
if not os.path.isdir(output_path):
    os.mkdir(output_path)

df.to_hdf(
    os.path.join(output_path, 'base_featurs_filled_by_matrix_cluster_2.h5'), 
    'base_featurs_filled_by_matrix_cluster_2')


In [58]:
# reload again for filling
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
# handling with infinity
df.replace([np.inf, -np.inf], np.nan, inplace = True)
for f_in_same_cluster in X_filled1.columns.tolist():
    print ('featurs', f_in_same_cluster)
    df[f_in_same_cluster] = X_filled1[f_in_same_cluster].tolist()
for f_in_same_cluster in X_filled2.columns.tolist():
    print ('featurs', f_in_same_cluster)
    df[f_in_same_cluster] = X_filled2[f_in_same_cluster].tolist()

(356251, 281)
featurs CODE_GENDER
featurs FLAG_DOCUMENT_3
featurs REGION_RATING_CLIENT
featurs REGION_RATING_CLIENT_W_CITY
featurs INCOME_TO_GOODS_PRICE_RATIO
featurs NAME_CONTRACT_TYPE_Cash loans
featurs NAME_EDUCATION_TYPE_Secondary / secondary special
featurs NAME_FAMILY_STATUS_Married
featurs NAME_HOUSING_TYPE_House / apartment
featurs NAME_INCOME_TYPE_Working
featurs PREV_APP_CREDIT_PERC_MAX
featurs PREV_APP_CREDIT_PERC_MEAN
featurs PREV_HUMAN_EVAL_MEAN
featurs PREV_HUMAN_EVAL_SUM
featurs PREV_NAME_CONTRACT_TYPE_Consumer loans_MEAN
featurs PREV_NAME_CONTRACT_STATUS_Approved_MEAN
featurs PREV_CODE_REJECT_REASON_XAP_MEAN
featurs PREV_NAME_PORTFOLIO_POS_MEAN
featurs APPROVED_APP_CREDIT_PERC_MIN
featurs APPROVED_HUMAN_EVAL_SUM
featurs POS_TIME_DECAYED_UNPAYED_RATIO_MAX
featurs POS_NAME_CONTRACT_STATUS_Active_MEAN
featurs INSTAL_PAYMENT_PERC_MEAN_MEAN
featurs INSTAL_PAYMENT_PERC_SUM_SUM
featurs AMT_CREDIT
featurs DAYS_BIRTH
featurs AMT_CREDIT_BY_NAME_EDUCATION_TYPE_OCCUPATION_TYPE
feat

In [62]:
df.to_hdf(
    os.path.join(output_path, 'base_featurs_filled_by_matrix_cluster_2_1.h5'), 
    'base_featurs_filled_by_matrix_cluster_2_1')


In [63]:
os.listdir('../features/filled_by_matrix_completion')

['base_featurs_filled_by_matrix_cluster_2_1.h5',
 'base_featurs_filled_by_matrix_cluster_2+1.h5',
 'base_featurs_filled_by_matrix_cluster_1.h5',
 'base_featurs_filled_by_matrix_cluster_2.h5']

# imputation - by KNN

In [26]:
# loading data
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
# handling with infinity
df.replace([np.inf, -np.inf], np.nan, inplace = True)
print (df.shape)


(356251, 281)
(356251, 280)


In [27]:
from ycimpute.imputer.knnimput import KNN # inputation library
X_missing_df = df[feature_clustering[feature_clustering.cluster == 1].features.tolist()]
print ('X_missing_df', X_missing_df.shape)
small_df_ls = []
num_split = 20
previous_step = None
for i, step in enumerate(np.arange(0, X_missing_df.shape[0], step = int(X_missing_df.shape[0] / num_split))):
    # for memory problem, we cannot feed all the data points into algorithm, which depends on row and n_features
    if i == 0:
        pass
    elif i == 1:
        small_df_ls.append(X_missing_df[0: step])
        previous_step = step
    elif i == (num_split):
        small_df_ls.append(X_missing_df[previous_step: ])
        break
    else:
        small_df_ls.append(X_missing_df[previous_step: step])
        previous_step = step
pd.concat(small_df_ls, axis = 0).shape

X_missing_df (356251, 24)


(356251, 24)

In [28]:
#-------------------
# imputating
#------------------
k = 65 # by cv
small_df_filled_ls = []
for small_df in small_df_ls:
    X_missing = small_df.copy()
    # core algorithm: input should be array
    X_filled = KNN(k = k).complete(X_missing.values)
    small_df_filled_ls.append(pd.DataFrame(X_filled, columns = X_missing.columns))


Imputing row 1/17812 with 0 missing, elapsed time: 84.634
Imputing row 101/17812 with 0 missing, elapsed time: 84.646
Imputing row 201/17812 with 0 missing, elapsed time: 84.656
Imputing row 301/17812 with 0 missing, elapsed time: 84.665
Imputing row 401/17812 with 0 missing, elapsed time: 84.675
Imputing row 501/17812 with 0 missing, elapsed time: 84.689
Imputing row 601/17812 with 0 missing, elapsed time: 84.701
Imputing row 701/17812 with 0 missing, elapsed time: 84.706
Imputing row 801/17812 with 0 missing, elapsed time: 84.719
Imputing row 901/17812 with 0 missing, elapsed time: 84.726
Imputing row 1001/17812 with 0 missing, elapsed time: 84.732
Imputing row 1101/17812 with 0 missing, elapsed time: 84.739
Imputing row 1201/17812 with 0 missing, elapsed time: 84.751
Imputing row 1301/17812 with 0 missing, elapsed time: 84.754
Imputing row 1401/17812 with 0 missing, elapsed time: 84.761
Imputing row 1501/17812 with 0 missing, elapsed time: 84.769
Imputing row 1601/17812 with 0 missi

Imputing row 13901/17812 with 0 missing, elapsed time: 85.873
Imputing row 14001/17812 with 14 missing, elapsed time: 85.879
Imputing row 14101/17812 with 0 missing, elapsed time: 85.888
Imputing row 14201/17812 with 0 missing, elapsed time: 85.898
Imputing row 14301/17812 with 0 missing, elapsed time: 85.909
Imputing row 14401/17812 with 0 missing, elapsed time: 85.916
Imputing row 14501/17812 with 0 missing, elapsed time: 85.923
Imputing row 14601/17812 with 0 missing, elapsed time: 85.929
Imputing row 14701/17812 with 0 missing, elapsed time: 85.942
Imputing row 14801/17812 with 0 missing, elapsed time: 85.949
Imputing row 14901/17812 with 0 missing, elapsed time: 85.959
Imputing row 15001/17812 with 0 missing, elapsed time: 85.969
Imputing row 15101/17812 with 0 missing, elapsed time: 85.972
Imputing row 15201/17812 with 0 missing, elapsed time: 85.976
Imputing row 15301/17812 with 0 missing, elapsed time: 85.985
Imputing row 15401/17812 with 0 missing, elapsed time: 85.995
Imputin

Imputing row 11101/17812 with 0 missing, elapsed time: 82.293
Imputing row 11201/17812 with 0 missing, elapsed time: 82.302
Imputing row 11301/17812 with 0 missing, elapsed time: 82.305
Imputing row 11401/17812 with 0 missing, elapsed time: 82.311
Imputing row 11501/17812 with 0 missing, elapsed time: 82.322
Imputing row 11601/17812 with 0 missing, elapsed time: 82.331
Imputing row 11701/17812 with 0 missing, elapsed time: 82.343
Imputing row 11801/17812 with 0 missing, elapsed time: 82.355
Imputing row 11901/17812 with 0 missing, elapsed time: 82.378
Imputing row 12001/17812 with 0 missing, elapsed time: 82.392
Imputing row 12101/17812 with 0 missing, elapsed time: 82.404
Imputing row 12201/17812 with 0 missing, elapsed time: 82.411
Imputing row 12301/17812 with 0 missing, elapsed time: 82.414
Imputing row 12401/17812 with 0 missing, elapsed time: 82.428
Imputing row 12501/17812 with 0 missing, elapsed time: 82.438
Imputing row 12601/17812 with 0 missing, elapsed time: 82.446
Imputing

Imputing row 7101/17812 with 0 missing, elapsed time: 82.732
Imputing row 7201/17812 with 0 missing, elapsed time: 82.735
Imputing row 7301/17812 with 0 missing, elapsed time: 82.740
Imputing row 7401/17812 with 14 missing, elapsed time: 82.744
Imputing row 7501/17812 with 0 missing, elapsed time: 82.748
Imputing row 7601/17812 with 0 missing, elapsed time: 82.756
Imputing row 7701/17812 with 0 missing, elapsed time: 82.759
Imputing row 7801/17812 with 0 missing, elapsed time: 82.766
Imputing row 7901/17812 with 0 missing, elapsed time: 82.774
Imputing row 8001/17812 with 0 missing, elapsed time: 82.780
Imputing row 8101/17812 with 0 missing, elapsed time: 82.784
Imputing row 8201/17812 with 0 missing, elapsed time: 82.791
Imputing row 8301/17812 with 14 missing, elapsed time: 82.798
Imputing row 8401/17812 with 0 missing, elapsed time: 82.805
Imputing row 8501/17812 with 0 missing, elapsed time: 82.810
Imputing row 8601/17812 with 0 missing, elapsed time: 82.815
Imputing row 8701/1781

Imputing row 5401/17812 with 0 missing, elapsed time: 88.476
Imputing row 5501/17812 with 0 missing, elapsed time: 88.481
Imputing row 5601/17812 with 0 missing, elapsed time: 88.490
Imputing row 5701/17812 with 0 missing, elapsed time: 88.496
Imputing row 5801/17812 with 0 missing, elapsed time: 88.500
Imputing row 5901/17812 with 0 missing, elapsed time: 88.508
Imputing row 6001/17812 with 0 missing, elapsed time: 88.512
Imputing row 6101/17812 with 0 missing, elapsed time: 88.520
Imputing row 6201/17812 with 4 missing, elapsed time: 88.525
Imputing row 6301/17812 with 0 missing, elapsed time: 88.536
Imputing row 6401/17812 with 0 missing, elapsed time: 88.539
Imputing row 6501/17812 with 0 missing, elapsed time: 88.542
Imputing row 6601/17812 with 0 missing, elapsed time: 88.549
Imputing row 6701/17812 with 0 missing, elapsed time: 88.555
Imputing row 6801/17812 with 0 missing, elapsed time: 88.563
Imputing row 6901/17812 with 0 missing, elapsed time: 88.570
Imputing row 7001/17812 

...

Imputing row 13701/17812 with 0 missing, elapsed time: 85.766
Imputing row 13801/17812 with 0 missing, elapsed time: 85.796
Imputing row 13901/17812 with 0 missing, elapsed time: 85.802
Imputing row 14001/17812 with 0 missing, elapsed time: 85.810
Imputing row 14101/17812 with 0 missing, elapsed time: 85.816
Imputing row 14201/17812 with 14 missing, elapsed time: 85.837
Imputing row 14301/17812 with 0 missing, elapsed time: 85.868
Imputing row 14401/17812 with 14 missing, elapsed time: 85.918
Imputing row 14501/17812 with 0 missing, elapsed time: 85.962
Imputing row 14601/17812 with 0 missing, elapsed time: 85.968
Imputing row 14701/17812 with 14 missing, elapsed time: 85.971
Imputing row 14801/17812 with 0 missing, elapsed time: 85.988
Imputing row 14901/17812 with 0 missing, elapsed time: 86.039
Imputing row 15001/17812 with 0 missing, elapsed time: 86.097
Imputing row 15101/17812 with 0 missing, elapsed time: 86.133
Imputing row 15201/17812 with 0 missing, elapsed time: 86.141
Imput

Imputing row 16901/17812 with 0 missing, elapsed time: 83.932
Imputing row 17001/17812 with 0 missing, elapsed time: 83.939
Imputing row 17101/17812 with 0 missing, elapsed time: 83.946
Imputing row 17201/17812 with 2 missing, elapsed time: 83.956
Imputing row 17301/17812 with 0 missing, elapsed time: 83.964
Imputing row 17401/17812 with 0 missing, elapsed time: 83.968
Imputing row 17501/17812 with 0 missing, elapsed time: 83.971
Imputing row 17601/17812 with 0 missing, elapsed time: 83.973
Imputing row 17701/17812 with 0 missing, elapsed time: 83.975
Imputing row 17801/17812 with 0 missing, elapsed time: 83.981
Imputing row 1/17812 with 0 missing, elapsed time: 87.562
Imputing row 101/17812 with 0 missing, elapsed time: 87.574
Imputing row 201/17812 with 0 missing, elapsed time: 87.586
Imputing row 301/17812 with 0 missing, elapsed time: 87.601
Imputing row 401/17812 with 0 missing, elapsed time: 87.610
Imputing row 501/17812 with 0 missing, elapsed time: 87.624
Imputing row 601/17812

Imputing row 13601/17812 with 0 missing, elapsed time: 88.381
Imputing row 13701/17812 with 0 missing, elapsed time: 88.387
Imputing row 13801/17812 with 0 missing, elapsed time: 88.394
Imputing row 13901/17812 with 0 missing, elapsed time: 88.402
Imputing row 14001/17812 with 0 missing, elapsed time: 88.412
Imputing row 14101/17812 with 0 missing, elapsed time: 88.416
Imputing row 14201/17812 with 0 missing, elapsed time: 88.417
Imputing row 14301/17812 with 0 missing, elapsed time: 88.426
Imputing row 14401/17812 with 0 missing, elapsed time: 88.431
Imputing row 14501/17812 with 0 missing, elapsed time: 88.434
Imputing row 14601/17812 with 10 missing, elapsed time: 88.442
Imputing row 14701/17812 with 0 missing, elapsed time: 88.451
Imputing row 14801/17812 with 0 missing, elapsed time: 88.460
Imputing row 14901/17812 with 0 missing, elapsed time: 88.465
Imputing row 15001/17812 with 0 missing, elapsed time: 88.468
Imputing row 15101/17812 with 0 missing, elapsed time: 88.475
Imputin

Imputing row 12301/17812 with 0 missing, elapsed time: 89.108
Imputing row 12401/17812 with 0 missing, elapsed time: 89.112
Imputing row 12501/17812 with 0 missing, elapsed time: 89.114
Imputing row 12601/17812 with 0 missing, elapsed time: 89.117
Imputing row 12701/17812 with 0 missing, elapsed time: 89.119
Imputing row 12801/17812 with 0 missing, elapsed time: 89.121
Imputing row 12901/17812 with 0 missing, elapsed time: 89.123
Imputing row 13001/17812 with 0 missing, elapsed time: 89.124
Imputing row 13101/17812 with 0 missing, elapsed time: 89.126
Imputing row 13201/17812 with 0 missing, elapsed time: 89.128
Imputing row 13301/17812 with 0 missing, elapsed time: 89.131
Imputing row 13401/17812 with 0 missing, elapsed time: 89.133
Imputing row 13501/17812 with 0 missing, elapsed time: 89.135
Imputing row 13601/17812 with 0 missing, elapsed time: 89.138
Imputing row 13701/17812 with 0 missing, elapsed time: 89.142
Imputing row 13801/17812 with 0 missing, elapsed time: 89.144
Imputing

Imputing row 17001/17812 with 0 missing, elapsed time: 92.057
Imputing row 17101/17812 with 0 missing, elapsed time: 92.059
Imputing row 17201/17812 with 0 missing, elapsed time: 92.063
Imputing row 17301/17812 with 0 missing, elapsed time: 92.064
Imputing row 17401/17812 with 0 missing, elapsed time: 92.065
Imputing row 17501/17812 with 0 missing, elapsed time: 92.066
Imputing row 17601/17812 with 0 missing, elapsed time: 92.067
Imputing row 17701/17812 with 0 missing, elapsed time: 92.068
Imputing row 17801/17812 with 0 missing, elapsed time: 92.071
Imputing row 1/17823 with 0 missing, elapsed time: 99.841
Imputing row 101/17823 with 0 missing, elapsed time: 99.843
Imputing row 201/17823 with 0 missing, elapsed time: 99.845
Imputing row 301/17823 with 0 missing, elapsed time: 99.847
Imputing row 401/17823 with 0 missing, elapsed time: 99.850
Imputing row 501/17823 with 0 missing, elapsed time: 99.855
Imputing row 601/17823 with 0 missing, elapsed time: 99.858
Imputing row 701/17823 w

In [29]:
X_filled1 = pd.concat(small_df_filled_ls, axis = 0)
X_filled1.shape

(356251, 24)

In [33]:
X_filled1.head()

,CODE_GENDER,FLAG_DOCUMENT_3,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,INCOME_TO_GOODS_PRICE_RATIO,NAME_CONTRACT_TYPE_Cash loans,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_HOUSING_TYPE_House / apartment,NAME_INCOME_TYPE_Working,...,PREV_NAME_CONTRACT_TYPE_Consumer loans_MEAN,PREV_NAME_CONTRACT_STATUS_Approved_MEAN,PREV_CODE_REJECT_REASON_XAP_MEAN,PREV_NAME_PORTFOLIO_POS_MEAN,APPROVED_APP_CREDIT_PERC_MIN,APPROVED_HUMAN_EVAL_SUM,POS_TIME_DECAYED_UNPAYED_RATIO_MAX,POS_NAME_CONTRACT_STATUS_Active_MEAN,INSTAL_PAYMENT_PERC_MEAN_MEAN,INSTAL_PAYMENT_PERC_SUM_SUM
0,0.0,1.0,2.0,2.0,0.576923,1.0,1.0,0.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,0.250000,1.000000,1.000000,19.0
1,1.0,1.0,1.0,1.0,0.239044,1.0,0.0,1.0,1.0,0.0,...,0.666667,1.000000,1.000000,0.666667,0.868825,3.0,0.001600,0.928571,1.000000,25.0
2,0.0,0.0,2.0,2.0,0.500000,0.0,1.0,0.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.207699,1.0,0.001372,0.750000,1.000000,3.0
3,1.0,1.0,2.0,2.0,0.454545,1.0,1.0,0.0,1.0,1.0,...,0.222222,0.555556,0.888889,0.222222,0.943934,5.0,0.250000,0.857143,1.000000,16.0
4,0.0,0.0,2.0,2.0,0.236842,1.0,1.0,0.0,1.0,1.0,...,0.333333,1.000000,1.000000,0.333333,0.791139,6.0,0.541667,0.939394,0.956044,63.0


In [31]:
from ycimpute.imputer.knnimput import KNN # inputation library
X_missing_df = df[feature_clustering[feature_clustering.cluster == 2].features.tolist()]
print ('X_missing_df', X_missing_df.shape)
small_df_ls = []
num_split = 20
previous_step = None
for i, step in enumerate(np.arange(0, X_missing_df.shape[0], step = int(X_missing_df.shape[0] / num_split))):
    # for memory problem, we cannot feed all the data points into algorithm, which depends on row and n_features
    if i == 0:
        pass
    elif i == 1:
        small_df_ls.append(X_missing_df[0: step])
        previous_step = step
    elif i == (num_split):
        small_df_ls.append(X_missing_df[previous_step: ])
        break
    else:
        small_df_ls.append(X_missing_df[previous_step: step])
        previous_step = step
#-------------------
# imputating
#------------------
k = 65 # by cv
small_df_filled_ls = []
for small_df in small_df_ls:
    X_missing = small_df.copy()
    # core algorithm: input should be array
    X_filled = KNN(k = k).complete(X_missing.values)
    small_df_filled_ls.append(pd.DataFrame(X_filled, columns = X_missing.columns))


X_missing_df (356251, 11)
Imputing row 1/17812 with 0 missing, elapsed time: 74.512
Imputing row 101/17812 with 0 missing, elapsed time: 74.560
Imputing row 201/17812 with 9 missing, elapsed time: 74.619
Imputing row 301/17812 with 0 missing, elapsed time: 74.655
Imputing row 401/17812 with 9 missing, elapsed time: 74.709
Imputing row 501/17812 with 9 missing, elapsed time: 74.775
Imputing row 601/17812 with 0 missing, elapsed time: 74.827
Imputing row 701/17812 with 0 missing, elapsed time: 74.864
Imputing row 801/17812 with 0 missing, elapsed time: 74.927
Imputing row 901/17812 with 9 missing, elapsed time: 74.982
Imputing row 1001/17812 with 9 missing, elapsed time: 75.026
Imputing row 1101/17812 with 0 missing, elapsed time: 75.081
Imputing row 1201/17812 with 9 missing, elapsed time: 75.129
Imputing row 1301/17812 with 9 missing, elapsed time: 75.169
Imputing row 1401/17812 with 0 missing, elapsed time: 75.222
Imputing row 1501/17812 with 0 missing, elapsed time: 75.279
Imputing r

Imputing row 13501/17812 with 0 missing, elapsed time: 81.809
Imputing row 13601/17812 with 0 missing, elapsed time: 81.858
Imputing row 13701/17812 with 0 missing, elapsed time: 81.916
Imputing row 13801/17812 with 0 missing, elapsed time: 81.982
Imputing row 13901/17812 with 9 missing, elapsed time: 82.034
Imputing row 14001/17812 with 0 missing, elapsed time: 82.103
Imputing row 14101/17812 with 0 missing, elapsed time: 82.143
Imputing row 14201/17812 with 0 missing, elapsed time: 82.213
Imputing row 14301/17812 with 9 missing, elapsed time: 82.278
Imputing row 14401/17812 with 0 missing, elapsed time: 82.332
Imputing row 14501/17812 with 0 missing, elapsed time: 82.390
Imputing row 14601/17812 with 0 missing, elapsed time: 82.464
Imputing row 14701/17812 with 9 missing, elapsed time: 82.524
Imputing row 14801/17812 with 0 missing, elapsed time: 82.573
Imputing row 14901/17812 with 0 missing, elapsed time: 82.631
Imputing row 15001/17812 with 0 missing, elapsed time: 82.691
Imputing

Imputing row 9101/17812 with 0 missing, elapsed time: 69.407
Imputing row 9201/17812 with 0 missing, elapsed time: 69.472
Imputing row 9301/17812 with 0 missing, elapsed time: 69.531
Imputing row 9401/17812 with 9 missing, elapsed time: 69.593
Imputing row 9501/17812 with 0 missing, elapsed time: 69.641
Imputing row 9601/17812 with 0 missing, elapsed time: 69.677
Imputing row 9701/17812 with 0 missing, elapsed time: 69.753
Imputing row 9801/17812 with 0 missing, elapsed time: 69.810
Imputing row 9901/17812 with 0 missing, elapsed time: 69.865
Imputing row 10001/17812 with 0 missing, elapsed time: 69.933
Imputing row 10101/17812 with 0 missing, elapsed time: 69.979
Imputing row 10201/17812 with 9 missing, elapsed time: 70.040
Imputing row 10301/17812 with 0 missing, elapsed time: 70.113
Imputing row 10401/17812 with 9 missing, elapsed time: 70.183
Imputing row 10501/17812 with 0 missing, elapsed time: 70.258
Imputing row 10601/17812 with 0 missing, elapsed time: 70.314
Imputing row 1070

Imputing row 4901/17812 with 0 missing, elapsed time: 78.317
Imputing row 5001/17812 with 9 missing, elapsed time: 78.358
Imputing row 5101/17812 with 0 missing, elapsed time: 78.400
Imputing row 5201/17812 with 0 missing, elapsed time: 78.429
Imputing row 5301/17812 with 9 missing, elapsed time: 78.469
Imputing row 5401/17812 with 0 missing, elapsed time: 78.499
Imputing row 5501/17812 with 0 missing, elapsed time: 78.535
Imputing row 5601/17812 with 0 missing, elapsed time: 78.569
Imputing row 5701/17812 with 0 missing, elapsed time: 78.611
Imputing row 5801/17812 with 0 missing, elapsed time: 78.650
Imputing row 5901/17812 with 0 missing, elapsed time: 78.686
Imputing row 6001/17812 with 0 missing, elapsed time: 78.722
Imputing row 6101/17812 with 0 missing, elapsed time: 78.756
Imputing row 6201/17812 with 9 missing, elapsed time: 78.793
Imputing row 6301/17812 with 0 missing, elapsed time: 78.830
Imputing row 6401/17812 with 9 missing, elapsed time: 78.872
Imputing row 6501/17812 

Imputing row 501/17812 with 0 missing, elapsed time: 84.613
Imputing row 601/17812 with 0 missing, elapsed time: 84.653
Imputing row 701/17812 with 0 missing, elapsed time: 84.679
Imputing row 801/17812 with 0 missing, elapsed time: 84.724
Imputing row 901/17812 with 0 missing, elapsed time: 84.760
Imputing row 1001/17812 with 0 missing, elapsed time: 84.805
Imputing row 1101/17812 with 0 missing, elapsed time: 84.846
Imputing row 1201/17812 with 0 missing, elapsed time: 84.885
Imputing row 1301/17812 with 0 missing, elapsed time: 84.917
Imputing row 1401/17812 with 9 missing, elapsed time: 84.956
Imputing row 1501/17812 with 0 missing, elapsed time: 84.992
Imputing row 1601/17812 with 9 missing, elapsed time: 85.029
Imputing row 1701/17812 with 0 missing, elapsed time: 85.065
Imputing row 1801/17812 with 0 missing, elapsed time: 85.109
Imputing row 1901/17812 with 0 missing, elapsed time: 85.136
Imputing row 2001/17812 with 0 missing, elapsed time: 85.166
Imputing row 2101/17812 with 

...

Imputing row 13101/17812 with 9 missing, elapsed time: 73.066
Imputing row 13201/17812 with 9 missing, elapsed time: 73.108
Imputing row 13301/17812 with 9 missing, elapsed time: 73.156
Imputing row 13401/17812 with 0 missing, elapsed time: 73.212
Imputing row 13501/17812 with 9 missing, elapsed time: 73.264
Imputing row 13601/17812 with 9 missing, elapsed time: 73.320
Imputing row 13701/17812 with 0 missing, elapsed time: 73.372
Imputing row 13801/17812 with 0 missing, elapsed time: 73.430
Imputing row 13901/17812 with 0 missing, elapsed time: 73.500
Imputing row 14001/17812 with 9 missing, elapsed time: 73.562
Imputing row 14101/17812 with 0 missing, elapsed time: 73.605
Imputing row 14201/17812 with 9 missing, elapsed time: 73.675
Imputing row 14301/17812 with 9 missing, elapsed time: 73.714
Imputing row 14401/17812 with 9 missing, elapsed time: 73.759
Imputing row 14501/17812 with 0 missing, elapsed time: 73.812
Imputing row 14601/17812 with 0 missing, elapsed time: 73.885
Imputing

Imputing row 8601/17812 with 0 missing, elapsed time: 70.427
Imputing row 8701/17812 with 9 missing, elapsed time: 70.477
Imputing row 8801/17812 with 0 missing, elapsed time: 70.535
Imputing row 8901/17812 with 0 missing, elapsed time: 70.601
Imputing row 9001/17812 with 9 missing, elapsed time: 70.677
Imputing row 9101/17812 with 0 missing, elapsed time: 70.731
Imputing row 9201/17812 with 9 missing, elapsed time: 70.786
Imputing row 9301/17812 with 9 missing, elapsed time: 70.845
Imputing row 9401/17812 with 0 missing, elapsed time: 70.915
Imputing row 9501/17812 with 9 missing, elapsed time: 70.961
Imputing row 9601/17812 with 9 missing, elapsed time: 71.016
Imputing row 9701/17812 with 0 missing, elapsed time: 71.094
Imputing row 9801/17812 with 9 missing, elapsed time: 71.170
Imputing row 9901/17812 with 0 missing, elapsed time: 71.219
Imputing row 10001/17812 with 9 missing, elapsed time: 71.269
Imputing row 10101/17812 with 0 missing, elapsed time: 71.334
Imputing row 10201/178

Imputing row 4401/17812 with 0 missing, elapsed time: 75.027
Imputing row 4501/17812 with 0 missing, elapsed time: 75.056
Imputing row 4601/17812 with 0 missing, elapsed time: 75.091
Imputing row 4701/17812 with 9 missing, elapsed time: 75.125
Imputing row 4801/17812 with 0 missing, elapsed time: 75.155
Imputing row 4901/17812 with 0 missing, elapsed time: 75.192
Imputing row 5001/17812 with 0 missing, elapsed time: 75.226
Imputing row 5101/17812 with 9 missing, elapsed time: 75.262
Imputing row 5201/17812 with 9 missing, elapsed time: 75.292
Imputing row 5301/17812 with 9 missing, elapsed time: 75.330
Imputing row 5401/17812 with 9 missing, elapsed time: 75.372
Imputing row 5501/17812 with 9 missing, elapsed time: 75.419
Imputing row 5601/17812 with 0 missing, elapsed time: 75.475
Imputing row 5701/17812 with 0 missing, elapsed time: 75.503
Imputing row 5801/17812 with 9 missing, elapsed time: 75.530
Imputing row 5901/17812 with 9 missing, elapsed time: 75.568
Imputing row 6001/17812 

Imputing row 1/17823 with 0 missing, elapsed time: 66.043
Imputing row 101/17823 with 9 missing, elapsed time: 66.088
Imputing row 201/17823 with 0 missing, elapsed time: 66.142
Imputing row 301/17823 with 9 missing, elapsed time: 66.198
Imputing row 401/17823 with 0 missing, elapsed time: 66.260
Imputing row 501/17823 with 0 missing, elapsed time: 66.297
Imputing row 601/17823 with 0 missing, elapsed time: 66.334
Imputing row 701/17823 with 0 missing, elapsed time: 66.369
Imputing row 801/17823 with 0 missing, elapsed time: 66.427
Imputing row 901/17823 with 0 missing, elapsed time: 66.495
Imputing row 1001/17823 with 0 missing, elapsed time: 66.571
Imputing row 1101/17823 with 9 missing, elapsed time: 66.641
Imputing row 1201/17823 with 9 missing, elapsed time: 66.721
Imputing row 1301/17823 with 0 missing, elapsed time: 66.765
Imputing row 1401/17823 with 0 missing, elapsed time: 66.805
Imputing row 1501/17823 with 9 missing, elapsed time: 66.877
Imputing row 1601/17823 with 0 missi

Imputing row 13401/17823 with 0 missing, elapsed time: 73.439
Imputing row 13501/17823 with 0 missing, elapsed time: 73.489
Imputing row 13601/17823 with 0 missing, elapsed time: 73.543
Imputing row 13701/17823 with 0 missing, elapsed time: 73.599
Imputing row 13801/17823 with 9 missing, elapsed time: 73.637
Imputing row 13901/17823 with 9 missing, elapsed time: 73.709
Imputing row 14001/17823 with 0 missing, elapsed time: 73.748
Imputing row 14101/17823 with 0 missing, elapsed time: 73.819
Imputing row 14201/17823 with 0 missing, elapsed time: 73.866
Imputing row 14301/17823 with 0 missing, elapsed time: 73.937
Imputing row 14401/17823 with 0 missing, elapsed time: 73.990
Imputing row 14501/17823 with 0 missing, elapsed time: 74.026
Imputing row 14601/17823 with 9 missing, elapsed time: 74.076
Imputing row 14701/17823 with 0 missing, elapsed time: 74.130
Imputing row 14801/17823 with 9 missing, elapsed time: 74.180
Imputing row 14901/17823 with 0 missing, elapsed time: 74.250
Imputing

In [32]:
X_filled2 = pd.concat(small_df_filled_ls, axis = 0)
X_filled2.shape

(356251, 11)

In [34]:
# reload again for filling
df = pd.read_hdf('../features/base_featurs.h5','base_featurs')
print (df.shape)
copy_for_the_following_merge = df[['SK_ID_CURR','TARGET']].copy()
no_need_to_inpute = ['index']
df.drop(no_need_to_inpute, axis = 1, inplace = True)
# handling with infinity
df.replace([np.inf, -np.inf], np.nan, inplace = True)

(356251, 281)


In [35]:
for f_in_same_cluster in X_filled1.columns.tolist():
    print ('featurs', f_in_same_cluster)
    df[f_in_same_cluster] = X_filled1[f_in_same_cluster].tolist()

featurs CODE_GENDER
featurs FLAG_DOCUMENT_3
featurs REGION_RATING_CLIENT
featurs REGION_RATING_CLIENT_W_CITY
featurs INCOME_TO_GOODS_PRICE_RATIO
featurs NAME_CONTRACT_TYPE_Cash loans
featurs NAME_EDUCATION_TYPE_Secondary / secondary special
featurs NAME_FAMILY_STATUS_Married
featurs NAME_HOUSING_TYPE_House / apartment
featurs NAME_INCOME_TYPE_Working
featurs PREV_APP_CREDIT_PERC_MAX
featurs PREV_APP_CREDIT_PERC_MEAN
featurs PREV_HUMAN_EVAL_MEAN
featurs PREV_HUMAN_EVAL_SUM
featurs PREV_NAME_CONTRACT_TYPE_Consumer loans_MEAN
featurs PREV_NAME_CONTRACT_STATUS_Approved_MEAN
featurs PREV_CODE_REJECT_REASON_XAP_MEAN
featurs PREV_NAME_PORTFOLIO_POS_MEAN
featurs APPROVED_APP_CREDIT_PERC_MIN
featurs APPROVED_HUMAN_EVAL_SUM
featurs POS_TIME_DECAYED_UNPAYED_RATIO_MAX
featurs POS_NAME_CONTRACT_STATUS_Active_MEAN
featurs INSTAL_PAYMENT_PERC_MEAN_MEAN
featurs INSTAL_PAYMENT_PERC_SUM_SUM


In [36]:
for f_in_same_cluster in X_filled2.columns.tolist():
    print ('featurs', f_in_same_cluster)
    df[f_in_same_cluster] = X_filled2[f_in_same_cluster].tolist()

featurs AMT_CREDIT
featurs DAYS_BIRTH
featurs AMT_CREDIT_BY_NAME_EDUCATION_TYPE_OCCUPATION_TYPE
featurs DAYS_BIRTH_BY_OCCUPATION_TYPE
featurs DAYS_BIRTH_BY_CODE_GENDER_OCCUPATION_TYPE
featurs DAYS_BIRTH_BY_REG_CITY_NOT_WORK_CITY_OCCUPATION_TYPE
featurs DAYS_BIRTH_BY_NAME_INCOME_TYPE_OCCUPATION_TYPE
featurs AMT_CREDIT_BY_NAME_INCOME_TYPE_OCCUPATION_TYPE
featurs AMT_GOODS_PRICE_BY_NAME_INCOME_TYPE_OCCUPATION_TYPE
featurs DAYS_BIRTH_BY_ORGANIZATION_TYPE_OCCUPATION_TYPE
featurs DAYS_REGISTRATION_BY_ORGANIZATION_TYPE_OCCUPATION_TYPE


In [37]:
import os
#-------------------
# save
#-------------------
output_path = '../features/filled_by_knn'
if not os.path.isdir(output_path):
    os.mkdir(output_path)

df.to_hdf(
    os.path.join(output_path, 'base_featurs_filled_knn_k_65_by_cluster_2+1.h5'), 
    'base_featurs_filled_knn_k_65_by_cluster_2+1')


/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'base_featurs_filled_knn_k_65_by_cluster_2+1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


In [20]:
df = pd.read_hdf('../features/filled_by_knn/base_featurs_filled_knn_k_65_by_cluster_0.h5')
df.drop(['TARGET'], axis = 1, inplace = True)
df = pd.merge(df, copy_for_the_following_merge, on = 'SK_ID_CURR', how = 'left')

In [23]:
df.TARGET

0         1.0
1         0.0
2         0.0
3         0.0
4         0.0
5         0.0
6         0.0
7         0.0
8         0.0
9         0.0
10        0.0
11        0.0
12        0.0
13        0.0
14        0.0
15        0.0
16        0.0
17        0.0
18        0.0
19        0.0
20        0.0
21        0.0
22        0.0
23        0.0
24        0.0
25        0.0
26        1.0
27        0.0
28        0.0
29        0.0
         ... 
356221    NaN
356222    NaN
356223    NaN
356224    NaN
356225    NaN
356226    NaN
356227    NaN
356228    NaN
356229    NaN
356230    NaN
356231    NaN
356232    NaN
356233    NaN
356234    NaN
356235    NaN
356236    NaN
356237    NaN
356238    NaN
356239    NaN
356240    NaN
356241    NaN
356242    NaN
356243    NaN
356244    NaN
356245    NaN
356246    NaN
356247    NaN
356248    NaN
356249    NaN
356250    NaN
Name: TARGET, Length: 356251, dtype: float64

# imputation - by MICE

In [13]:
X_missing = df[feature_clustering[feature_clustering.cluster == 0].features.tolist()]

In [14]:
X_missing.shape

(356251, 244)

(356251, 280)

In [16]:
# 這個實驗需要10個小時啊.....
#-------------------
# core algorithm: input should be array
#-------------------
from fancyimpute import MICE # for imputing

logging.info('visit_sequence: {}'.format('monotone')) 
logging.info('impute_type: {}'.format('col')) 
logging.info('init_fill_method: {}'.format('mean')) 
logging.info('target == 1')
X_filled = MICE(visit_sequence = 'monotone', 
                impute_type = 'col',
                 init_fill_method = 'mean').complete(X_missing.values)
    

[MICE] Completing matrix with shape (356251, 244)
[MICE] Starting imputation round 1/110, elapsed time 1.201
[MICE] Starting imputation round 2/110, elapsed time 377.596
[MICE] Starting imputation round 3/110, elapsed time 756.648
[MICE] Starting imputation round 4/110, elapsed time 1128.018
[MICE] Starting imputation round 5/110, elapsed time 1501.540


KeyboardInterrupt: 

In [27]:
X_filled_copy_2 = pd.DataFrame(X_filled, columns = X_missing.columns)
print (X_filled_copy_2.shape)
for f_in_same_cluster in X_filled_copy_2.columns.tolist():
    print ('featurs', f_in_same_cluster)
    df[f_in_same_cluster] = X_filled_copy_2[f_in_same_cluster]
    

(356251, 24)
featurs CODE_GENDER
featurs FLAG_DOCUMENT_3
featurs REGION_RATING_CLIENT
featurs REGION_RATING_CLIENT_W_CITY
featurs INCOME_TO_GOODS_PRICE_RATIO
featurs NAME_CONTRACT_TYPE_Cash loans
featurs NAME_EDUCATION_TYPE_Secondary / secondary special
featurs NAME_FAMILY_STATUS_Married
featurs NAME_HOUSING_TYPE_House / apartment
featurs NAME_INCOME_TYPE_Working
featurs PREV_APP_CREDIT_PERC_MAX
featurs PREV_APP_CREDIT_PERC_MEAN
featurs PREV_HUMAN_EVAL_MEAN
featurs PREV_HUMAN_EVAL_SUM
featurs PREV_NAME_CONTRACT_TYPE_Consumer loans_MEAN
featurs PREV_NAME_CONTRACT_STATUS_Approved_MEAN
featurs PREV_CODE_REJECT_REASON_XAP_MEAN
featurs PREV_NAME_PORTFOLIO_POS_MEAN
featurs APPROVED_APP_CREDIT_PERC_MIN
featurs APPROVED_HUMAN_EVAL_SUM
featurs POS_TIME_DECAYED_UNPAYED_RATIO_MAX
featurs POS_NAME_CONTRACT_STATUS_Active_MEAN
featurs INSTAL_PAYMENT_PERC_MEAN_MEAN
featurs INSTAL_PAYMENT_PERC_SUM_SUM


In [28]:
X_filled_copy_1 = pd.DataFrame(X_filled, columns = X_missing.columns)
print (X_filled_copy_1.shape)
for f_in_same_cluster in X_filled_copy_1.columns.tolist():
    print ('featurs', f_in_same_cluster)
    df[f_in_same_cluster] = X_filled_copy_1[f_in_same_cluster]
    

(356251, 24)
featurs CODE_GENDER
featurs FLAG_DOCUMENT_3
featurs REGION_RATING_CLIENT
featurs REGION_RATING_CLIENT_W_CITY
featurs INCOME_TO_GOODS_PRICE_RATIO
featurs NAME_CONTRACT_TYPE_Cash loans
featurs NAME_EDUCATION_TYPE_Secondary / secondary special
featurs NAME_FAMILY_STATUS_Married
featurs NAME_HOUSING_TYPE_House / apartment
featurs NAME_INCOME_TYPE_Working
featurs PREV_APP_CREDIT_PERC_MAX
featurs PREV_APP_CREDIT_PERC_MEAN
featurs PREV_HUMAN_EVAL_MEAN
featurs PREV_HUMAN_EVAL_SUM
featurs PREV_NAME_CONTRACT_TYPE_Consumer loans_MEAN
featurs PREV_NAME_CONTRACT_STATUS_Approved_MEAN
featurs PREV_CODE_REJECT_REASON_XAP_MEAN
featurs PREV_NAME_PORTFOLIO_POS_MEAN
featurs APPROVED_APP_CREDIT_PERC_MIN
featurs APPROVED_HUMAN_EVAL_SUM
featurs POS_TIME_DECAYED_UNPAYED_RATIO_MAX
featurs POS_NAME_CONTRACT_STATUS_Active_MEAN
featurs INSTAL_PAYMENT_PERC_MEAN_MEAN
featurs INSTAL_PAYMENT_PERC_SUM_SUM


In [29]:
print (df.shape)
#-------------------
# save
#-------------------
output_path = '../features/filled_by_mice'
if not os.path.isdir(output_path):
    os.mkdir(output_path)

df.to_hdf(
    os.path.join(output_path, 'base_featurs_filled_mice_by_cluster_2+1.h5'), 
    'base_featurs_filled_mice_by_cluster_1')



(356251, 280)


In [30]:
os.listdir('../features/filled_by_mice')

['base_featurs_filled_mice_by_cluster_2+1.h5',
 'base_featurs_filled_mice_by_cluster_2.h5',
 'base_featurs_filled_mice_clustering.h5',
 'base_featurs_filled_mice_by_cluster_1.h5',
 'base_featurs_filled_mice_half_training_data.h5']